In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
# Copy Files From External Drive To Local Disk

from data.file_utils import Files
import shutil
import os
import numpy as np

db_folder = "/Volumes/DIY_NVME/Bio-informatic project/Exp_20240213_N2_JC01/"
# raw_ds_folder = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_01_PARTIAL2/"
raw_ds_folder = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_01_RAND/"

file_iter = Files(db_folder)
print(len(file_iter))

video_length = 1000
sample_start_list = range(8000, 58000, 3700)
rnd_fids = np.random.choice(len(file_iter), size=1000, replace=False)

# for i in rnd_fids:
#     src = file_iter.seek(i)
#     filename = file_iter.get_filename()
#     dst = raw_ds_folder + filename
#     shutil.copyfile(src, dst)

# for vid_start in sample_start_list:
#     file_iter.seek(vid_start-1)
#     ret = input(f"Move {video_length} files, strating after {file_iter.get_filename()}? [y/n]")
#     if ret.upper() != 'Y':
#         break
#     for a in range(video_length):
#         src = file_iter.__next__()
#         filename = file_iter.get_filename()
#         dst = raw_ds_folder + filename
#         # os.symlink(src, dst)
#         shutil.copyfile(src, dst)





In [ ]:
cv.destroyAllWindows()

In [ ]:
def image_transform(image: np.ndarray):
    # remove gaussian noise
    image = cv.GaussianBlur(image, (3, 3), 0)

    # quantize image
    N = 50
    image = np.round(image * (N / 255), decimals=0) * (255 / N)

    return image.astype(np.uint8)

In [1]:
from dataclasses import dataclass
# class to hold the db paths of our pipeline 
@dataclass(frozen=True)
class PATHS:
    raw_db:str
    cropped_db:str
    dlc_db:str


In [2]:
# define the paths

db_paths =  PATHS(raw_db="/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_02_PARTIAL/",
            cropped_db="/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_02_EXTRACTED/",
            dlc_db="/Users/guycohen/Desktop/Test-Dataset-2024-02-26/labeled-data/output/",
            )
db_paths2 =  PATHS(raw_db="/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_01_RAND/",
            cropped_db="/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_01_EXTRACTED2/",
            dlc_db="/Users/guycohen/Desktop/Test-Dataset-2024-02-26/labeled-data/output/",
            )

In [3]:
# Initialize frame reader and extractor

from data.frame_reader import FrameReader
from data.video_extractor import VideoExtractor


reader = FrameReader.create_from_directory(db_paths2.raw_db)
extractor = VideoExtractor(reader, num_workers=4)

In [6]:
sample_path = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/samples"

extractor.background()

extractor.create_samples(1000, (400,400), sample_path)

In [7]:
reader = FrameReader.create_from_directory(sample_path)
for f in reader:
    if reader.frame_shape != (400,400):
        print("ERROR")

In [ ]:
from dataset.raw_dataset import *
from data.file_utils import pickle_load_object, pickle_save_object

background = extractor.background()

h, w = reader.frame_shape
exp_meta = ExperimentMeta(60, (w, h), 133.33333, background)
experiment = ExperimentDataset.create_from_frame_reader(reader, exp_meta)

In [6]:
from data.file_utils import folder_convert_images, rename_raw_file_names, Files
import os

# sample_path = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/samples/"

# file_iter = Files(sample_path, 'png')

# for file in file_iter:
#     # print(file[:-len(file_iter.get_filename())])
#     new_name = file_iter.get_filename().split('-')[-1]
#     dst = sample_path + new_name
#     os.rename(file, dst)



In [ ]:
# Copy the extracted images to the dlc labeling folder

from data.file_utils import Files
import shutil

every_x_samples = 30
count = 0

directories = Files(db_paths.cropped_db, scan_dirs=True)
for dir in directories:
    file_iter = Files(dir, '.png')
    count += len(file_iter)
    for i in range(0, len(file_iter), every_x_samples):
        file = file_iter.seek(i)
        shutil.copyfile(file, db_paths.dlc_db+file_iter.get_filename())

print(count)

In [ ]:
from view_controller import ViewController

view = ViewController(FrameReader.create_from_directory("sample_all_0"))

while True:
    #view.move_position(5, 5)
    view.visualize_world()
    cv.imshow("micro", view.camera_view())
    cv.waitKey(0)

    if view.progress() == False:
        break
    
cv.destroyAllWindows()

In [ ]:
raise Exception("Finished")

In [ ]:
# TODO: IMPROVE
def find_boxes(image: np.ndarray) -> np.ndarray:
    # increase contrast of original image
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # remove gaussian noise
    smoothed = cv.GaussianBlur(image, (3, 3), 0)

    # apply threshold to image to convert it into black & white
    _, mask = cv.threshold(smoothed, 160, 255, cv.THRESH_BINARY_INV)

    # do some morphological magic to clean up noise from the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)

    # dilate to increase all object sizes in the mask
    kernel = np.ones((3, 3), np.uint8)
    mask = cv.dilate(mask, kernel, iterations=5)

    image[mask == 0] = 255
    cv.imshow("masked", image)

    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Populate bounding boxes
    bbox_list = []
    for c in contours:
        area = cv.contourArea(c)

        if area < 500 or area > 4000:
            continue

        box = cv.boundingRect(c)
        bbox_list.append(box)

    # Turn our bboxes into 2d ndarray
    bboxes = np.asanyarray(bbox_list)
    return bboxes

In [ ]:
from view_controller import VideoStream
from data.legacy_box_extractor import BoxExtractor

video = VideoStream("worms.avi", padding_size=(251, 251))
extractor = BoxExtractor(find_boxes, slice_size=251, object_size=150)

for frame in video:
    if cv.waitKey(1) & 0xFF == ord("q"):
        break

    head_coords, rois = extractor.run(frame)

    # Draw bboxes
    for box in head_coords:
        x, y, w, h = box
        cv.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 3)

    cv.imshow("BBOXES", frame)

In [ ]:
import math
from view_controller import ViewController

sim = ViewController("worms.avi", color_conversion=cv.COLOR_BGR2GRAY)

fps = 40
micro_view_time = 200
tracking_time = 50
moving_time = 25

frame_time = 1 / fps
micro_view_frames = math.ceil(micro_view_time / frame_time)
tracking_frames = math.ceil(tracking_time / frame_time)
moving_frames = math.ceil(moving_time / frame_time)

while True:
    # Here we assume that the worm is kinda centered on the screen
    sim.micro_view()  # Takes 'micro_frames' frames, can do other stuff meanwhile
    sim.next_frame(micro_view_frames - tracking_frames)

    cam_view = sim.camera_view()
    dx, dy = real_time_tracking(cam_view)  # Takes 'track_frames' frames, can't do anything meanwhile
    sim.next_frame(tracking_frames)  # micro and tracking finished

    sim.next_frame(moving_frames)  # moving takes 'move_frames' frames, can do other stuff meanwhile
    sim.move_position(dx, dy)